# Find Best Optimizer

### Step 1: change the following and what i will mention

In [1]:
datapathway=r"C:\Users\wingt\Downloads\S11_along_BD_205.csv"
xstart=3            #column of xstart
xstop=5            #column of xend
ystart=6           #column of ystart
yname='Stress'      # name of the ultimate output array
yparameter='Position'       #position here but it will be what is the parameter to be tuned
xcolumn=['Travel_length', 'Welding_speed', 'Net_energy_input'] #name of inputs

### Step 2 import libraries

In [2]:
import numpy as np
import pandas as pd
import warnings
import os
import random
import pickle
# import ML related libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from scipy import interpolate

### Step 3: Tune the Optimizer-function
a function to tune the hyperparameter of the ANN machine-> optimizer
model used is from KerasRegressor
Used gridsearch cross validation
printing the best optimizer name and the score of this machine
printing the mean_test_score, std_test_score, parameter for all the optimizer tested

In [3]:
def tune_optimizer():
    callbacks = [EarlyStopping(monitor='mse', patience=100, verbose=0),]
    model = KerasRegressor(build_fn=create_model, nb_epoch=400, batch_size=10, verbose=0) 
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'] # 200 hidden nodes
    param_grid = dict(optimizer=optimizer) 
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8)
    grid_result = grid.fit(OP_X_train, OP_Y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
    for index, value in enumerate(grid_result.cv_results_['mean_test_score']):
        print("%f (%f) with: %r" % (grid_result.cv_results_['mean_test_score'][index], grid_result.cv_results_['std_test_score'][index], grid_result.cv_results_['params'][index]))
    print(grid_result.scorer_)
    print(pd.DataFrame(grid_result.cv_results_,index=optimizer))

### Step 4: import data + preprocessing(assign x and y values)
import the data using pd.read_csv
only get columns required
assign column names
get y data for training as well

In [4]:
def data_import(csv_file_name):
    '''
    used to import dataset and split training and test dataset
    csv_file_name is the dateset root
    reture traing and test datasets
    '''
    raw_data = pd.read_csv(csv_file_name,header=None).dropna()
    x = raw_data.iloc[:,xstart:(xstop+1)]
    #x.columns = xcolumn
    y = raw_data.iloc[:,ystart:]
    return x, y

### Step 5: function for creating a model for tuning optimiser

In [5]:
def create_model(optimizer='adam'):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    model = keras.Sequential([
        keras.layers.Dense(200, activation=tf.nn.leaky_relu, input_shape=(4,)),
        keras.layers.Dense(21,activation='linear')
    ])
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
    model.summary()
    return model

### Step 6: Uniform Spaced Sampling
Actually this is not required in our case but for this database I will just keep it first 

In [6]:
def uniformly_spaced_sampling(y_label, y_label_new, y):
    '''
    fit and interpolate
    '''
    f = interpolate.interp1d(y_label, y, kind='linear') # linear interpolation function use first row of y and all of y -> linear interpolate location and y value
    ynew=pd.DataFrame(f(y_label_new)) #first row of y (linear distributed) -> linearly interpolate(predict) when in evenly distributed location, what r y
    return ynew

### Step 7: assign column names to data set 
* turn a lot of y columns into one y column only (for ANN2)
* However it is not helping us to find the correlations between output and inputs (no equations)

In [7]:
def data_melt(x, y):
    '''
    Add position as input
    input:
        x: old input 
        y: old output
    output:
        x_new: new input 
        y_new: new output
    '''
    y_label_str = [str(x) for x in y_label_new] # make a list of string of y_label_new
    dataset = pd.concat([x, y],axis=1, ignore_index=True)
    col_names = xcolumn + y_label_str
    dataset.columns = col_names
    dataset = dataset.melt(id_vars=xcolumn, 
        var_name=yparameter, 
        value_name=yname) #for given welding parameter, in a given location, the stress is in this dataset
    x_new = dataset.iloc[:, 0:(len(xcolumn)+1)]
    y_new = dataset.iloc[:, (len(xcolumn)+1)]
    return x_new, y_new

### Step 8: Define a function getting y(output) label
* use y in the csv
* the first row of that csv is y_lab
* y_lab_new is min to max of y_lab in length of y_lab
* this is also not really required for our database (should be idk)
* for our database, we should know the name of all the parameters to be calibrated

In [8]:
def get_y_lable(root):
    '''
    Create an array for uniform interval depth 
    (Used biased mesh in simulaiton, so the sample point depth is not uniform) 
    input:
        root: file root contains the sample point depth information
    outputs:
        y_lab: old sample point position array
        y_lab_new: new sample point position array
    '''
    x, y = data_import(root)
    y_lab = y.iloc[0, :]
    y_lab_new = np.linspace(round(min(y_lab),2), 
                              round(max(y_lab),2), 
                              round(len(y_lab),2))
    return y_lab, y_lab_new
    

### Step 9: Prepocessing function
(Bascially just for outputting a standardised form of x and y)

In [9]:
def OP_pre_processing(model_type,x,y):
    '''
    Data preprocessing (Uniformly spaced sampling, normalisation, train test split)
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)        
        x: a dataframe of inputs of the whole dataset
        y: a dataframe of outputs of the whole dataset
    outputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        Y_test: Test output
    '''
      # uniformly_spaced_sampling
    global y_label, y_label_new
    y_label, y_label_new = get_y_lable(r'C:\Users\wingt\Downloads\benchmark_BD1.csv')
    y = uniformly_spaced_sampling(y_label, y_label_new, y)
    # data reconstruction
    if 'ANN2' in model_type:
        X_train, Y_train = data_melt(x, y) 
    # Normalization
    global scaler_X, scaler_Y
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    scaled_train_X = scaler_X.fit_transform(X_train)
    if Y_train.ndim == 1:
        Y_train = np.array(Y_train).reshape(-1,1)#column array
        scaled_train_Y = scaler_Y.fit_transform(Y_train)
        OP_X_train = scaled_train_X
        OP_Y_train = scaled_train_Y

    return model_type, OP_X_train, OP_Y_train

### Step 10: fix random seed

In [10]:
def seed_tensorflow(seed):
    '''
    Fix ramdom seed
    '''
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
# Fix ramdom seed
warnings.filterwarnings('ignore')
seed_tensorflow(42)

### Step 11: import data

In [11]:
# Import dataset
x, y = data_import(datapathway)
# Pre-processing

### Step 12: Optimizer performance maximizing data-preprocess

In [12]:
model_type, OP_X_train, OP_Y_train= OP_pre_processing('ANN2',x,y)

### Step 13: Actual Tuning

In [13]:
tune_optimizer()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1000      
_________________________________________________________________
dense_1 (Dense)              (None, 21)                4221      
Total params: 5,221
Trainable params: 5,221
Non-trainable params: 0
_________________________________________________________________
Best: -0.173304 using {'optimizer': 'Adamax'}
-0.596253 (0.289181) with: {'optimizer': 'SGD'}
-0.233960 (0.142055) with: {'optimizer': 'RMSprop'}
-0.921237 (0.424848) with: {'optimizer': 'Adagrad'}
-0.984558 (0.425031) with: {'optimizer': 'Adadelta'}
-0.223719 (0.113217) with: {'optimizer': 'Adam'}
-0.173304 (0.057913) with: {'optimizer': 'Adamax'}
-0.222976 (0.123062) with: {'optimizer': 'Nadam'}
<function _passthrough_scorer at 0x000002A72936CF70>
          mean_fit_time  std_fit_time  mean_score_time  std_sc